In [14]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.model_selection import train_test_split
from aif360.algorithms.preprocessing import Reweighing
from sklearn.base import BaseEstimator, ClassifierMixin
from aif360.sklearn.inprocessing import ExponentiatedGradientReduction
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import LinearSVC
from aif360.algorithms.postprocessing import EqOddsPostprocessing 
from sklearn.metrics import accuracy_score, recall_score, f1_score

In [15]:
def compute_performance_metrics(y_test, y_pred, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1_score_value = f1_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} Recall: {recall}")
    print(f"{model_name} F1 Score: {f1_score_value}")

def compute_classification_metric(dataset, predictions, label_name_v, favorable_label_v, unfavorable_label_v, privileged_attribute, unprivileged_attributes):
    features = [privileged_attribute] + unprivileged_attributes

    # Rimuovi valori mancanti
    dataset = dataset.dropna()
    predictions = predictions.dropna()

    aif_race_dataset = BinaryLabelDataset(
        df=dataset,
        favorable_label=favorable_label_v,
        unfavorable_label=unfavorable_label_v,
        label_names=[label_name_v],
        protected_attribute_names=features
    )

    aif_race_pred = BinaryLabelDataset(
        df=predictions,
        favorable_label=favorable_label_v,
        unfavorable_label=unfavorable_label_v,
        label_names=[label_name_v],
        protected_attribute_names=features
    )

    race_privileged_group = [{privileged_attribute: 1}]
    race_unprivileged_groups = [{attr: 1} for attr in unprivileged_attributes]

    fairness_metrics = ClassificationMetric(
        dataset=aif_race_dataset,
        classified_dataset=aif_race_pred,
        unprivileged_groups=race_unprivileged_groups,
        privileged_groups=race_privileged_group
    )
    
    return fairness_metrics

def compute_fairness_metrics(fairness_metrics: ClassificationMetric):
    # Values less than 0 indicate that privileged group has higher
    # proportion of predicted positive outcomes than unprivileged group.
    # Value higher than 0 indicates that unprivileged group has higher proportion
    # of predicted positive outcomes than privileged group.
    SPD = round(fairness_metrics.statistical_parity_difference(),3)

    # Measures the deviation from the equality of opportunity, which means that the same
    # proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
    EOD = round(fairness_metrics.equal_opportunity_difference(),3)

    # Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
    # A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
    # groups have the same probability of being classified positively.
    AOD = round(fairness_metrics.average_odds_difference(),3)

    print(f"Statistical Parity Difference (SPD): {SPD}")
    print(f"Average Odds Difference (AOD): {AOD}")
    print(f"Equal Opportunity Difference (EOD): {EOD}")

In [16]:
dataset_path = 'adult/tidyAdult-trained.csv'
df_raw = pd.read_csv(dataset_path)
pd.set_option('display.max_columns', None)
df_raw.head()

,age,fnlwgt,education-num,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_11th,education_12th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Prof-school,education_Some-college,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native-country_Cambodia,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,sex_Female
0,50,288,13,13,0,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,38,464,9,40,0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,53,484,7,40,0,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,28,581,13,40,0,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,Fal

In [17]:
X = df_raw.drop(columns="income")
y = df_raw["income"]

privileged_attributeSex = "sex_Male"
unprivileged_attributesSex = ["sex_Female"]
sex_features = [privileged_attributeSex] + unprivileged_attributesSex

privileged_attributeRace = "race_White"
unprivileged_attributesRace = ["race_Black", "race_Other", "race_Asian-Pac-Islander"]
race_features = [privileged_attributeRace] + unprivileged_attributesRace

In [18]:
# Define four sets and apply the function
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, # 0.2 indicates a test set size of 20%
                                                    random_state=42)

In [19]:
# Add your functions here
# compute_performance_metrics and compute_classification_metric
# Add the definition for compute_fairness_metrics

# Train the Decision Tree classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

# Predizione sui dati di test
dt_predictions = dt_clf.predict(X_test)

# Calcolo delle metriche di performance per il Decision Tree grezzo
compute_performance_metrics(y_test, dt_predictions, "Decision Tree")

# Aggiunta della colonna del target al dataframe di X_test e rimozione di valori mancanti
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
dataset = dataset.dropna()

# Conversione dei dati in formato BinaryLabelDataset
test_dataset = BinaryLabelDataset(df=dataset, label_names=['income'], protected_attribute_names=sex_features)

# Creazione di un dataset di predizioni per il post-processing
predictions = dataset.copy(deep=True)
predictions['income'] = dt_predictions
predictions = predictions.dropna()

test_pred_dataset = BinaryLabelDataset(df=predictions, label_names=['income'], protected_attribute_names=sex_features)

# Applicazione del post-processing con EqOddsPostprocessing
eq_odds = EqOddsPostprocessing(privileged_groups=[{sex_features[0]: 1}], unprivileged_groups=[{sex_features[0]: 0}])
eq_odds = eq_odds.fit(test_dataset, test_pred_dataset)

# Fai delle predizioni post-processate
eq_odds_pred = eq_odds.predict(test_pred_dataset)
eq_odds_labels = eq_odds_pred.labels

# Calcolo delle metriche di performance per il modello post-processato
compute_performance_metrics(y_test, eq_odds_labels, "Decision Tree Post-processed")


Decision Tree Accuracy: 0.798811330298432
Decision Tree Recall: 0.48350253807106597
Decision Tree F1 Score: 0.4892455858747994
Decision Tree Post-processed Accuracy: 0.7716236722306525
Decision Tree Post-processed Recall: 0.4860406091370558
Decision Tree Post-processed F1 Score: 0.4589574595566207


In [20]:
# Creazione del dataset di test con le etichette originali e predizioni del modello
test_dataset_with_labels = X_test.copy(deep=True)
test_dataset_with_labels['income'] = y_test

# Creazione del dataset di predizioni del modello post-processato
predictions_post_processed = X_test.copy(deep=True)
predictions_post_processed['income'] = eq_odds_labels

In [21]:
# Calcolo delle metriche di fairness
fairness_metrics = compute_classification_metric(test_dataset_with_labels,predictions_post_processed,'income',1,0,privileged_attributeSex,unprivileged_attributesSex)
compute_fairness_metrics(fairness_metrics)


Statistical Parity Difference (SPD): -0.055
Average Odds Difference (AOD): 0.011
Equal Opportunity Difference (EOD): 0.024


In [22]:
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))

# Addestra il classificatore sui dati di training
svm_classifier.fit(X_train, y_train)

# After the training phase, the model will be tested by predicting the values on the test set
svm_scores = svm_classifier.decision_function(X_test)
svm_prediction = svm_classifier.predict(X_test)

compute_performance_metrics(y_test,svm_prediction,"SVM")

# Aggiunta della colonna del target al dataframe di X_test e rimozione di valori mancanti
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
dataset = dataset.dropna()

# Conversione dei dati in formato BinaryLabelDataset
test_dataset = BinaryLabelDataset(df=dataset, label_names=['income'], protected_attribute_names=sex_features)

# Creazione di un dataset di predizioni per il post-processing
predictions = dataset.copy(deep=True)
predictions['income'] = svm_prediction
predictions = predictions.dropna()

test_pred_dataset = BinaryLabelDataset(df=predictions, label_names=['income'], protected_attribute_names=sex_features)

# Applicazione del post-processing con EqOddsPostprocessing
eq_odds = EqOddsPostprocessing(privileged_groups=[{sex_features[0]: 1}], unprivileged_groups=[{sex_features[0]: 0}])
eq_odds = eq_odds.fit(test_dataset, test_pred_dataset)

# Fai delle predizioni post-processate
eq_odds_pred = eq_odds.predict(test_pred_dataset)
eq_odds_labels = eq_odds_pred.labels

# Calcolo delle metriche di performance per il modello post-processato
compute_performance_metrics(y_test, eq_odds_labels, "SVM Post-processed")

SVM Accuracy: 0.8373798684876075
SVM Recall: 0.39847715736040606
SVM F1 Score: 0.4940991345397325
SVM Post-processed Accuracy: 0.8166413758219524
SVM Post-processed Recall: 0.38451776649746194
SVM Post-processed F1 Score: 0.4552967693463561


In [23]:
# Creazione del dataset di test con le etichette originali e predizioni del modello
test_dataset_with_labels = X_test.copy(deep=True)
test_dataset_with_labels['income'] = y_test

# Creazione del dataset di predizioni del modello post-processato
predictions_post_processed = X_test.copy(deep=True)
predictions_post_processed['income'] = eq_odds_labels

In [24]:
# Calcolo delle metriche di fairness
fairness_metrics = compute_classification_metric(test_dataset_with_labels,predictions_post_processed,'income',1,0,privileged_attributeSex,unprivileged_attributesSex)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.053
Average Odds Difference (AOD): -0.002
Equal Opportunity Difference (EOD): -0.005


In [25]:
rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)

compute_performance_metrics(y_test,rf_predictions,"Random Forest")

# Aggiunta della colonna del target al dataframe di X_test e rimozione di valori mancanti
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
dataset = dataset.dropna()

# Conversione dei dati in formato BinaryLabelDataset
test_dataset = BinaryLabelDataset(df=dataset, label_names=['income'], protected_attribute_names=sex_features)

# Creazione di un dataset di predizioni per il post-processing
predictions = dataset.copy(deep=True)
predictions['income'] = rf_predictions
predictions = predictions.dropna()

test_pred_dataset = BinaryLabelDataset(df=predictions, label_names=['income'], protected_attribute_names=sex_features)


# Applicazione del post-processing con EqOddsPostprocessing
eq_odds = EqOddsPostprocessing(privileged_groups=[{sex_features[0]: 1}], unprivileged_groups=[{sex_features[0]: 0}])
eq_odds = eq_odds.fit(test_dataset, test_pred_dataset)

# Fai delle predizioni post-processate
eq_odds_pred = eq_odds.predict(test_pred_dataset)
eq_odds_labels = eq_odds_pred.labels

# Calcolo delle metriche di performance per il modello post-processato
compute_performance_metrics(y_test, eq_odds_labels, "Random Forest Post-processed")

Random Forest Accuracy: 0.842185128983308
Random Forest Recall: 0.4930203045685279
Random Forest F1 Score: 0.5546038543897216
Random Forest Post-processed Accuracy: 0.8170207384926657
Random Forest Post-processed Recall: 0.48413705583756345
Random Forest Post-processed F1 Score: 0.5132862428523377


In [26]:
# Creazione del dataset di test con le etichette originali e predizioni del modello
test_dataset_with_labels = X_test.copy(deep=True)
test_dataset_with_labels['income'] = y_test

# Creazione del dataset di predizioni del modello post-processato
predictions_post_processed = X_test.copy(deep=True)
predictions_post_processed['income'] = eq_odds_labels

In [27]:
# Calcolo delle metriche di fairness
fairness_metrics = compute_classification_metric(test_dataset_with_labels,predictions_post_processed,'income',1,0,privileged_attributeSex,unprivileged_attributesSex)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.065
Average Odds Difference (AOD): -0.007
Equal Opportunity Difference (EOD): -0.017


LAVORO SU RACE

In [28]:
# Add your functions here
# compute_performance_metrics and compute_classification_metric
# Add the definition for compute_fairness_metrics

# Train the Decision Tree classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

# Predizione sui dati di test
dt_predictions = dt_clf.predict(X_test)

# Calcolo delle metriche di performance per il Decision Tree grezzo
compute_performance_metrics(y_test, dt_predictions, "Decision Tree")

# Aggiunta della colonna del target al dataframe di X_test e rimozione di valori mancanti
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
dataset = dataset.dropna()

# Conversione dei dati in formato BinaryLabelDataset
test_dataset = BinaryLabelDataset(df=dataset, label_names=['income'], protected_attribute_names=race_features)

# Creazione di un dataset di predizioni per il post-processing
predictions = dataset.copy(deep=True)
predictions['income'] = dt_predictions
predictions = predictions.dropna()

test_pred_dataset = BinaryLabelDataset(df=predictions, label_names=['income'], protected_attribute_names=race_features)

# Applicazione del post-processing con EqOddsPostprocessing
eq_odds = EqOddsPostprocessing(privileged_groups=[{race_features[0]: 1}], unprivileged_groups=[{race_features[0]: 0}])
eq_odds = eq_odds.fit(test_dataset, test_pred_dataset)

# Fai delle predizioni post-processate
eq_odds_pred = eq_odds.predict(test_pred_dataset)
eq_odds_labels = eq_odds_pred.labels

# Calcolo delle metriche di performance per il modello post-processato
compute_performance_metrics(y_test, eq_odds_labels, "Decision Tree Post-processed")


Decision Tree Accuracy: 0.798811330298432
Decision Tree Recall: 0.48350253807106597
Decision Tree F1 Score: 0.4892455858747994
Decision Tree Post-processed Accuracy: 0.7945118866970157
Decision Tree Post-processed Recall: 0.4302030456852792
Decision Tree Post-processed F1 Score: 0.4548809124454881


In [29]:
# Creazione del dataset di test con le etichette originali e predizioni del modello
test_dataset_with_labels = X_test.copy(deep=True)
test_dataset_with_labels['income'] = y_test

# Creazione del dataset di predizioni del modello post-processato
predictions_post_processed = X_test.copy(deep=True)
predictions_post_processed['income'] = eq_odds_labels

In [30]:
# Calcolo delle metriche di fairness
fairness_metrics = compute_classification_metric(test_dataset_with_labels,predictions_post_processed,'income',1,0,privileged_attributeRace,unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)


Statistical Parity Difference (SPD): -0.029
Average Odds Difference (AOD): 0.011
Equal Opportunity Difference (EOD): 0.024


In [31]:
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))

# Addestra il classificatore sui dati di training
svm_classifier.fit(X_train, y_train)

# After the training phase, the model will be tested by predicting the values on the test set
svm_scores = svm_classifier.decision_function(X_test)
svm_prediction = svm_classifier.predict(X_test)

compute_performance_metrics(y_test,svm_prediction,"SVM")

# Aggiunta della colonna del target al dataframe di X_test e rimozione di valori mancanti
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
dataset = dataset.dropna()

# Conversione dei dati in formato BinaryLabelDataset
test_dataset = BinaryLabelDataset(df=dataset, label_names=['income'], protected_attribute_names=race_features)

# Creazione di un dataset di predizioni per il post-processing
predictions = dataset.copy(deep=True)
predictions['income'] = svm_prediction
predictions = predictions.dropna()

test_pred_dataset = BinaryLabelDataset(df=predictions, label_names=['income'], protected_attribute_names=race_features)

# Applicazione del post-processing con EqOddsPostprocessing
eq_odds = EqOddsPostprocessing(privileged_groups=[{race_features[0]: 1}], unprivileged_groups=[{race_features[0]: 0}])
eq_odds = eq_odds.fit(test_dataset, test_pred_dataset)

# Fai delle predizioni post-processate
eq_odds_pred = eq_odds.predict(test_pred_dataset)
eq_odds_labels = eq_odds_pred.labels

# Calcolo delle metriche di performance per il modello post-processato
compute_performance_metrics(y_test, eq_odds_labels, "SVM Post-processed")

SVM Accuracy: 0.8373798684876075
SVM Recall: 0.39847715736040606
SVM F1 Score: 0.4940991345397325
SVM Post-processed Accuracy: 0.8340920586747598
SVM Post-processed Recall: 0.3965736040609137
SVM Post-processed F1 Score: 0.4879000780640125


In [32]:
# Creazione del dataset di test con le etichette originali e predizioni del modello
test_dataset_with_labels = X_test.copy(deep=True)
test_dataset_with_labels['income'] = y_test

# Creazione del dataset di predizioni del modello post-processato
predictions_post_processed = X_test.copy(deep=True)
predictions_post_processed['income'] = eq_odds_labels

In [33]:
# Calcolo delle metriche di fairness
fairness_metrics = compute_classification_metric(test_dataset_with_labels,predictions_post_processed,'income',1,0,privileged_attributeRace,unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.028
Average Odds Difference (AOD): -0.001
Equal Opportunity Difference (EOD): -0.006


In [34]:
rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)

compute_performance_metrics(y_test,rf_predictions,"Random Forest")

# Aggiunta della colonna del target al dataframe di X_test e rimozione di valori mancanti
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
dataset = dataset.dropna()

# Conversione dei dati in formato BinaryLabelDataset
test_dataset = BinaryLabelDataset(df=dataset, label_names=['income'], protected_attribute_names=race_features)

# Creazione di un dataset di predizioni per il post-processing
predictions = dataset.copy(deep=True)
predictions['income'] = rf_predictions
predictions = predictions.dropna()

test_pred_dataset = BinaryLabelDataset(df=predictions, label_names=['income'], protected_attribute_names=race_features)


# Applicazione del post-processing con EqOddsPostprocessing
eq_odds = EqOddsPostprocessing(privileged_groups=[{race_features[0]: 1}], unprivileged_groups=[{race_features[0]: 0}])
eq_odds = eq_odds.fit(test_dataset, test_pred_dataset)

# Fai delle predizioni post-processate
eq_odds_pred = eq_odds.predict(test_pred_dataset)
eq_odds_labels = eq_odds_pred.labels

# Calcolo delle metriche di performance per il modello post-processato
compute_performance_metrics(y_test, eq_odds_labels, "Random Forest Post-processed")

Random Forest Accuracy: 0.842185128983308
Random Forest Recall: 0.4930203045685279
Random Forest F1 Score: 0.5546038543897216
Random Forest Post-processed Accuracy: 0.8329539706626201
Random Forest Post-processed Recall: 0.4479695431472081
Random Forest Post-processed F1 Score: 0.5166483717526528


In [35]:
# Creazione del dataset di test con le etichette originali e predizioni del modello
test_dataset_with_labels = X_test.copy(deep=True)
test_dataset_with_labels['income'] = y_test

# Creazione del dataset di predizioni del modello post-processato
predictions_post_processed = X_test.copy(deep=True)
predictions_post_processed['income'] = eq_odds_labels

In [36]:
# Calcolo delle metriche di fairness
fairness_metrics = compute_classification_metric(test_dataset_with_labels,predictions_post_processed,'income',1,0,privileged_attributeRace,unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.033
Average Odds Difference (AOD): 0.003
Equal Opportunity Difference (EOD): 0.005
